### 需要安装CPLEX
* [IBM CPLEX](http://www-03.ibm.com/software/products/en/ibmilogcpleoptistud)

In [10]:
#encoding=utf-8
# -*- coding: utf-8 -*-
# 基本部件的引用声明
import sys
import re
import codecs
import os
import shutil
import cplex
import jieba
import graphlab
import numpy as np
from array import array
from collections import Counter

# reload(sys)
# sys.setdefaultencoding('utf-8')

### 载入数据

In [11]:
# graphlab.load_sframe('.') 


newses = graphlab.load_sframe('./news_finished')
weibos = graphlab.load_sframe('./weibo_parsed')
# weibos_wvec = graphlab.load_sframe('./weibo_wordvec')

### 线性规划模型
<img src="http://o9206epbs.bkt.clouddn.com/linear.png" width="300px">  

其中, `Gaini(b)`, `ncnd(b)`和`Z(s)`是变量(variables), 需要cplex根据模型计算最优值

### 工具函数
* 计算bi-gram
* 分句
* 计算倒数和

In [50]:
def read_from_file(path):
    with open(path,"r") as fp:
        words = fp.read()
    return words

def get_stop_words(filepath):
    words = read_from_file(filepath)
    result = jieba.cut(words)
    new_words = []
    for r in result:
        new_words.append(r)
    return set(new_words)

def remove_stop_words(words,stop_words_set):
    new_words = []
    for w in words:
        if w not in stop_words_set:
            new_words.append(w)
    return new_words

def cal_bigram(text):
    stop_set = get_stop_words('./stop.txt')
    parsed_text = remove_stop_words(jieba.cut(text , cut_all = False) , stop_set)
    words = [w.strip() for w in parsed_text if w.strip()]
    counter = Counter()
    for i in range(len(words) - 1):
        w = words[i] + words[i + 1]
        counter[w] = counter[w] + 1
    bidict = dict(counter)
    return bidict

def split_by_set(text):
    return re.split('。|！|【|】|#' , text)

def calc_const(weibo_bidict):
    return 1.0 / np.sum(weibo_bidict.values())

### 预处理
将新闻和微博分句, 并计算词向量和bigram常数

In [79]:
# bigram_news = graphlab.SFrame()

def pre_process(news):
    sentences = split_by_set(news['text'])
    cplex_sentences = []
    cplex_bigram = Counter()
    cplex_weibo = []
    for assign_weibo in news['assign_weibos'][0:1]:
        weibo = weibos.filter_by(assign_weibo['id'], 'id')
#         print weibo['text'][0]
        
        bigram = cal_bigram(weibo['text'][0])
        cplex_weibo.append({'bigram': bigram, 'const': calc_const(bigram)})
        sentences.extend(split_by_set(weibo['text'][0]))
    for sentence in sentences:
        if len(sentence):
#             parsed_text = [t + '1' for t in jieba.cut(sentence, cut_all = False)]
#             parsed_text = [re.sub(u' |\t|　|，|；|,|、|…|“|”|"|', '', t) for t in jieba.cut(sentence, cut_all = False)]
#             for text in parsed_text:
#                 print text
            bigram = cal_bigram(sentence)
            cplex_bigram = cplex_bigram + Counter(bigram)
            cplex_sentences.append({'sentence': sentence, 'bigram': bigram})
        
    return {'sentences': cplex_sentences, 'bigram': cplex_bigram, 'weibo': cplex_weibo}
    

### 测试
测试以上函数正确性

In [80]:
news = pre_process(newses[2])
for sentence in news['sentences']:
    print "========== 句子 =========="
    print sentence['sentence']
    print "========== 词向量 =========="
    for word in sentence['bigram']:
        print word + u" 出现 " + str(sentence['bigram'][word]) + u" 次"
# for weibo in news['weibo']:
#     print "========== 微博 =========="
#     for word in weibo['bigram']:
#         print word + u" 出现 " + str(weibo['bigram'][word]) + u" 次"
#     print "========== 常数 =========="
#     print weibo['const']
# print "========== 总词向量 =========="
# for word in news['bigram']:
#     print word + u" 出现 " + str(news['bigram'][word]) + u" 次"

========== 句子 ==========
　　新华社无锡6月4日体育专电(记者 王镜宇 王恒志)国家体育总局棋牌运动管理中心党委书记、国际围棋联盟事务总长杨俊安4日在这里透露，如果不出意外柯洁九段将在年内进行和“阿尔法狗”的围棋“终极人机大战”
========== 词向量 ==========
新华社无锡 出现 1 次
记者王镜宇 出现 1 次
联盟事务 出现 1 次
总长杨俊 出现 1 次
事务总长 出现 1 次
运动管理中心 出现 1 次
人机大战 出现 1 次
国家体育总局棋牌 出现 1 次
管理中心党委书记 出现 1 次
月体育 出现 1 次
不出意外柯洁 出现 1 次
安透露 出现 1 次
九段年内 出现 1 次
王镜宇王恒志 出现 1 次
阿尔法狗 出现 1 次
柯洁九段 出现 1 次
狗围棋 出现 1 次
杨俊安 出现 1 次
围棋终极 出现 1 次
透露不出意外 出现 1 次
体育专电 出现 1 次
围棋联盟 出现 1 次
国际围棋 出现 1 次
党委书记国际 出现 1 次
终极人机 出现 1 次
年内阿尔法 出现 1 次
王恒志国家体育总局 出现 1 次
棋牌运动 出现 1 次
无锡月 出现 1 次
专电记者 出现 1 次
========== 句子 ==========
　　在4日下午举行的第37届世界业余围棋锦标赛新闻发布会上，杨俊安透露了这一消息
========== 词向量 ==========
37届 出现 1 次
新闻发布会 出现 1 次
业余围棋 出现 1 次
锦标赛新闻 出现 1 次
下午37 出现 1 次
世界业余 出现 1 次
发布会杨俊 出现 1 次
安透露 出现 1 次
围棋锦标赛 出现 1 次
届世界 出现 1 次
透露消息 出现 1 次
杨俊安 出现 1 次
========== 句子 ==========
据他介绍，中国围棋协会和“阿尔法狗”的团队就此事进行了接触和沟通，双方都有意向促成这项对抗
========== 词向量 ==========
团队此事 出现 1 次
中国围棋协会阿尔法 出现 1 次
促成这项 出现 1 次
此事接触 出现 1 次
意向促成 出现 1 次
接触沟通 出现 1 次
介绍中国围棋协会 出现 1 次
狗团队 出现 1 次
阿尔法狗 出现 1 次
沟通意向 出现 1 次
这项对抗 

In [81]:
def solve_cplex(prob):

    prob.solve()
    
    

#     numrows = prob.linear_constraints.get_num()
#     numcols = prob.variables.get_num()

#     print
    # solution.get_status() returns an integer code
#     print "Solution status = " , prob.solution.get_status(), ":",
    # the following line prints the corresponding string
#     print prob.solution.status[prob.solution.get_status()]
#     print "Solution value  = ", prob.solution.get_objective_value()


In [98]:
max_L = 300 # 选取句子的最大长度

def get_N_weibo(news, i, b):
    key = news['bigram'].keys()[b]
    if news['weibo'][i]['bigram'].has_key(key):
        return news['weibo'][i]['bigram'][key]
    else:
        return 0
    
def get_N_sentence(news, s, b):
    key = news['bigram'].keys()[b]
    if news['sentences'][s]['bigram'].has_key(key):
        return news['sentences'][s]['bigram'][key]
    else:
        return 0

def cplex_process(news):
    c = cplex.Cplex()
    c.objective.set_sense(c.objective.sense.maximize)
    
    c.set_log_stream(None)
    c.set_error_stream(None)
    c.set_warning_stream(None)
    c.set_results_stream(None)
    
    num_b = len(news['bigram'])
#     print 'size of b: %d' % num_b
    num_i = len(news['weibo'])
#     print 'size of i: %d' % num_i
    num_s = len(news['sentences'])
#     print 'size of s: %d' % num_s
    
#     z_vec = [0] * num_s
    z_type = [c.variables.type.binary] * num_s
    z_name = ['z_' + str(s) for s in range(num_s)]
    z_obj = [0] * num_s
#     z_ub = [1] * num_s
#     z_lb = [0] * num_s
    
#     gain_mat = [0] * num_i * num_b
    gain_type = [c.variables.type.integer] * num_i * num_b
    gain_name = ['gain_' + str(i) + '_' + str(b) for i in range(num_i) for b in range(num_b)]
    gain_ub = [get_N_weibo(news, i, b) for i in range(num_i) for b in range(num_b)] # Gain < Ntwt
    gain_lb = [0] * num_i * num_b
    gain_obj = [news['weibo'][i]['const'] for i in range(num_i) for b in range(num_b)] # 目标函数, 取最大值
    
#     c.variables.add(ub = z_ub, lb = z_lb, names = z_name, types = z_type)
    c.variables.add(obj = z_obj, names = z_name, types = z_type)
    c.variables.add(obj = gain_obj, ub = gain_ub, lb = gain_lb, names = gain_name, types = gain_type)
    
    c1_rows = [] # Gain < Ncnd
    for b in range(num_b):
        for i in range(num_i):
            names = ['gain_' + str(i) + '_' + str(b)]
            values = [1.0]
            for s in range(num_s):
                names.append('z_' + str(s))
                values.append(-1 * get_N_sentence(news, s, b))
            c1_rows.append([names, values])
    c1_senses = 'L' * num_i * num_b
    c1_rhs = [0.0] * num_i * num_b
    
    c2_rows = [[['z_' + str(s) for s in range(num_s)], [len(news['sentences'][s]['sentence']) for s in range(num_s)]]] # sum(Z(s) * length(s)) < L
    c2_senses = 'L'
    c2_rhs = [max_L]
    
    c.linear_constraints.add(lin_expr = c1_rows, senses = c1_senses, rhs = c1_rhs)
    c.linear_constraints.add(lin_expr = c2_rows, senses = c2_senses, rhs = c2_rhs)
    
    c.solve()
    
    z_result = c.solution.get_values(z_name)
#     print z_result
    sentence_result = []
    for i in range(len(z_result)):
        if z_result[i]:
            sentence_result.append(news['sentences'][i]['sentence'])
            
    return sentence_result
            


### 测试

In [99]:
news = pre_process(newses[0])
cplex_process(news)

['\xe8\xaf\xb7\xe6\x94\xb6\xe8\x97\x8f\xe8\xbf\x99\xe4\xbb\xbd\xe9\xab\x98\xe8\x80\x83\xe5\xa4\x87\xe5\xbf\x98\xe5\xbd\x95',
 '\xe5\xb0\xb1\xe5\x9c\xa8\xe8\x80\x83\xe7\x94\x9f\xe4\xbb\xac\xe6\x8a\x93\xe7\xb4\xa7\xe5\xa4\x8d\xe4\xb9\xa0\xe3\x80\x81\xe6\x9c\x80\xe5\x90\x8e\xe5\x86\xb2\xe5\x88\xba\xe6\x97\xb6\xef\xbc\x8c\xe5\xaa\x92\xe4\xbd\x93\xe4\xb8\x93\xe9\x97\xa8\xe4\xb8\xba\xe9\xab\x98\xe8\x80\x83\xe5\xad\xa6\xe5\xad\x90\xe4\xbb\xac\xe5\x88\xb6\xe4\xbd\x9c\xe4\xba\x86\xe4\xb8\x80\xe4\xbb\xbd\xe9\xab\x98\xe8\x80\x83\xe5\xa4\x87\xe5\xbf\x98\xe5\xbd\x95\xef\xbc\x8c\xe5\x8c\x85\xe6\x8b\xac\xe8\x80\x83\xe8\xaf\x95\xe6\x97\xb6\xe9\x97\xb4\xe3\x80\x81\xe5\xa4\xa9\xe6\xb0\x94\xe6\x83\x85\xe5\x86\xb5\xe7\xad\x89\xe7\xad\x89\xe2\x80\xa6\xe2\x80\xa6\xe5\xa6\x82\xe6\x9e\x9c\xe4\xbd\xa0\xe5\xae\xb6\xe9\x87\x8c\xe6\x9c\x89\xe8\x80\x83\xe7\x94\x9f\xef\xbc\x8c\xe6\x88\x96\xe8\x80\x85\xe8\xba\xab\xe8\xbe\xb9\xe6\x9c\x89\xe8\x80\x83\xe7\x94\x9f\xef\xbc\x8c\xe5\xb0\xb1\xe8\xb5\xb6\xe5\xbf\xab\xe8\xbd\

### 正式处理

In [105]:
result = []
linen = {0,1,2,3,4,5,6,7,8,9,19,39,59,99,199,399,599,799,999,1299,1499,1699,1999,2499,2999,3499,3999,4499,4999,5999,8888,9999}
for i in range(len(newses)):
    news = pre_process(newses[i])
    try:
        result.append(cplex_process(news))
        if i in linen or i == len(newses) - 1:
            print "第 %d 条新闻的summary计算完成" % (i + 1)
    except cplex.exceptions.CplexSolverError, exc:
        print 'news %d failed with exception: ' % i
        print exc
        result.append([])
        
newses['summary'] = result
print "新闻summary计算全部完成"

第 1 条新闻的summary计算完成
第 2 条新闻的summary计算完成
第 3 条新闻的summary计算完成
第 4 条新闻的summary计算完成
第 5 条新闻的summary计算完成
第 6 条新闻的summary计算完成
第 7 条新闻的summary计算完成
第 8 条新闻的summary计算完成
第 9 条新闻的summary计算完成
第 10 条新闻的summary计算完成
第 20 条新闻的summary计算完成
news 21 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

news 34 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

第 40 条新闻的summary计算完成
news 57 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

第 60 条新闻的summary计算完成
news 81 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

news 83 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

news 87 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

news 96 failed with exception: 
CPLEX Error  1016: Promotional version. Problem size limits exceeded.

第 100 条新闻的summa

In [106]:
newses.save('./news_finished_summary')

In [114]:
print '========== 新闻原文 =========='
print newses['text'][1]
print '========== summary ==========='
# print ' '.join(newses['summary'][1])
for sentence in newses['summary'][1]:
    print sentence

========== 新闻原文 ==========
从昨天（3日）开始，在上海，有一群人冒着雨，在上海影院门口等待着什么，就像这样：到了今早（4日）7：30分左右，这支队伍变得更长了：原来，第19届上海国际电影节于今天上午8时正式开票，他们都是前来买票的观众。排在队伍最前的是孙小姐，她从昨天下午2点就开始等待，她告诉东方网记者：今年最想看的是日本影片：《和母亲一起生活》、《暗杀教室》等，为了这些难得一见的电影，排18个小时的队也是值得的。还有一位70岁高龄的“铁杆影迷”方先生，他从第一届上海电影节开始每年都会第一时间来现场买票。今天，方先生赶乘地铁首班车早早到达现场，他告诉东方网记者：自己这几天一直都在作“功课”，列出了一份长长的电影节心愿片单，今年最想看的是好莱坞影片《乔布斯》。在网络上，另一群人也是早早定好8：00分的闹钟，打算通过淘票票APP来订票。每经小编（微信号：nbdnews）了解到，6月11日到19日，第19届上海国际电影节将举行。日前公布了此次展映的完整片单及排片表，这次参加展映的影片有近600部。不仅有在大银幕上极为罕见的大师之作、今年戛纳电影节的入围电影，还有年轻人喜欢的日韩片，《哈利·波特》系列八部连映、莎翁影展、迪斯尼·皮克斯电影周、007回顾等，被影迷称为“上影节史上最强片单”。其中，电影节推出的“安德烈·塔可夫斯基回顾展”非常重磅，《伊万的童年》等多部名作都将在电影节公映。这些片子在上海的放映场次将超过1250场。这是部分将会在电影节期间放映的影片海报：此时的上海影城，已经接近9点，百米长队迟迟得不到动弹。▲聚集在现场的影迷一边等待，一边用手机刷票（图片来源：东方网）原来，影院和淘票票APP采用的是同一个售票系统后台，因此大批聚集在影院现场的影迷依然无法购票。另据中国青年网消息：铁杆影迷徐先生表示，他就是因为担心购票网站“秒杀”和网络崩溃问题，才想着自己现场来买踏实些，可以根据购票情况随时做出调整。“没想到今年系统崩溃，一直等到现在都没买到。”心疼70岁高龄的方先生，遇到了19年一次的“系统崩溃”……▲排在最前面的两个姑娘，买到了心仪的电影票。图片来源：新民晚报到了9：12分，第一位排队的影迷孙小姐才拿到了她的第一张票，她已经排了19个小时的队。在10：30分，情况有所好转，网站终于开始运行，但当网络购票的观众看到座位表的时候，差点吐血

In [115]:
newses.export_csv('./news_summary.csv')